In [1]:
from pygam import LinearGAM, s, f

In [2]:
import dask.dataframe as ddf
import pandas as pd
import datetime
import functools
import seaborn as sns
import matplotlib.pyplot as plt

### Import suicide death data

In [3]:
years = [year for year in range(1960, 2020)]
years

[1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [4]:
data = []
for year in years:
    suicide_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/Suicide_Death_NCHS/suicide_patient_level_1960_2020/overall/overall_suicide_patient_level_' + str(year) + '.csv', dtype = {'year': int, 'county_residence': str, 'county_death': str, 'state_residence': str, 'state_death': str})
    suicide_year = suicide_year.loc[:, ~suicide_year.columns.str.contains('^Unnamed')]
    data.append(suicide_year)

In [5]:
suicide = pd.concat(data)
suicide.head()

,year,county_residence,state_residence,state_death,county_death,month,sex,race,age,age_range,death_cause,date
0,1960,23001,23,23,23001,Dec,Male,White,7,age 25-64,nonfirearm_suicide,NaN
1,1960,48201,48,48,48201,Dec,Male,Black,10,65 and older,nonfirearm_suicide,NaN
2,1960,06037,06,06,06037,Apr,Female,White,8,age 25-64,nonfirearm_suicide,NaN
3,1960,48441,48,48,48441,Nov,Male,White,7,age 25-64,nonfirearm_suicide,NaN
4,1960,34013,34,34,34013,Feb,Male,White,8,age 25-64,nonfirearm_suicide,NaN


In [6]:
suicide = suicide.drop(['county_residence', 'state_residence'], axis = 1)
suicide = suicide.rename(columns = {'state_death': 'statefips', 'county_death': 'fips'})
suicide.head()

,year,statefips,fips,month,sex,race,age,age_range,death_cause,date
0,1960,23,23001,Dec,Male,White,7,age 25-64,nonfirearm_suicide,NaN
1,1960,48,48201,Dec,Male,Black,10,65 and older,nonfirearm_suicide,NaN
2,1960,06,06037,Apr,Female,White,8,age 25-64,nonfirearm_suicide,NaN
3,1960,48,48441,Nov,Male,White,7,age 25-64,nonfirearm_suicide,NaN
4,1960,34,34013,Feb,Male,White,8,age 25-64,nonfirearm_suicide,NaN


In [7]:
suicide

,year,statefips,fips,month,sex,race,age,age_range,death_cause,date
0,1960,23,23001,Dec,Male,White,7,age 25-64,nonfirearm_suicide,NaN
1,1960,48,48201,Dec,Male,Black,10,65 and older,nonfirearm_suicide,NaN
2,1960,06,06037,Apr,Female,White,8,age 25-64,nonfirearm_suicide,NaN
3,1960,48,48441,Nov,Male,White,7,age 25-64,nonfirearm_suicide,NaN
4,1960,34,34013,Feb,Male,White,8,age 25-64,nonfirearm_suicide,NaN
...,...,...,...,...,...,...,...,...,...,...
47669,2019,36,36081,Dec,Male,Other,5,age 25-64,nonfirearm_suicide,NaN
47670,2019,36,36081,Dec,Male,Black,5,age 25-64,nonfirearm_suicide,NaN
47671,2019,36,36005,Dec,Female,Black,4,age 0-24,nonfirearm_suicide,NaN
47672,2019,36,36005,Dec,Male,White,6,age 25-64,nonfirearm_suicide,NaN


### Load population data

In [8]:
data_pop = []
for year in years:
    pop_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/temp_bins_suicide/Population/population_monthly/population_monthly_' + str(year) + '.csv', dtype = {'year': int, 'fips': str})
    pop_year = pop_year.loc[:, ~pop_year.columns.str.contains('^Unnamed')]
    data_pop.append(pop_year)

In [9]:
pop = pd.concat(data_pop)
pop.head()

,year,fips,pop,month
0,1960,01001,18686,Jan
1,1960,01001,18691,Feb
2,1960,01001,18696,Mar
3,1960,01001,18701,Apr
4,1960,01001,18705,May


In [10]:
month_replace = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
                'Nov': 11, 'Dec': 12}
pop['month'] = pop['month'].replace(month_replace)
pop['month'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [11]:
pop_year = pop[pop['month'] == 12]
pop_year = pop_year.drop('month', axis = 1)
pop_year.head()

,year,fips,pop
11,1960,01001,18739
23,1960,01003,49088
35,1960,01005,24700
47,1960,01007,14357
59,1960,01009,25449


In [13]:
suicide = suicide.drop('date', axis = 1)
suicide.head()

,year,statefips,fips,month,sex,race,age,age_range,death_cause
0,1960,23,23001,Dec,Male,White,7,age 25-64,nonfirearm_suicide
1,1960,48,48201,Dec,Male,Black,10,65 and older,nonfirearm_suicide
2,1960,06,06037,Apr,Female,White,8,age 25-64,nonfirearm_suicide
3,1960,48,48441,Nov,Male,White,7,age 25-64,nonfirearm_suicide
4,1960,34,34013,Feb,Male,White,8,age 25-64,nonfirearm_suicide


In [14]:
suicide_death = suicide.drop(['sex', 'race', 'age', 'age_range', 'month'], axis = 1)
suicide_death = suicide_death.groupby(['year', 'statefips', 'fips']).count()
suicide_death = suicide_death.reset_index()
suicide_death.head()

,year,statefips,fips,death_cause
0,1960,00,00nan,32
1,1960,01,01003,1
2,1960,01,01007,2
3,1960,01,01009,2
4,1960,01,01011,1


In [15]:
suicide_death['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019])

In [16]:
suicide_pop = suicide_death.merge(pop_year, on = ['year', 'fips'], how = 'inner')
suicide_pop.head()

,year,statefips,fips,death_cause,pop
0,1960,01,01003,1,49088
1,1960,01,01007,2,14357
2,1960,01,01009,2,25449
3,1960,01,01011,1,13462
4,1960,01,01013,3,24560


In [17]:
state = pd.read_excel('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/temp_bins_suicide/test_paper/state_code.xlsx', engine='openpyxl', dtype = {'Code': int})
state.head()

,Code,State,Abbreviation,Alpha code
0,1,Alabama,Ala.,AL
1,2,Alaska,NaN,AK
2,4,Arizona,Ariz.,AZ
3,5,Arkansas,Ark.,AR
4,6,California,Calif.,CA


In [18]:
def convertStateCode(code):
    return str(code).rjust(2, '0')

In [19]:
state['Code'] = state['Code'].apply(convertStateCode)
state = state.rename(columns = {'Code': 'statefips', 'State': 'state_name', 'Alpha code': 'state'})
state.head()

,statefips,state_name,Abbreviation,state
0,01,Alabama,Ala.,AL
1,02,Alaska,NaN,AK
2,04,Arizona,Ariz.,AZ
3,05,Arkansas,Ark.,AR
4,06,California,Calif.,CA


In [20]:
west_states = ['Colorado', 'Wyoming', 'Montana', 'Idaho', 'Washington',
              'Oregon', 'Utah', 'Nevada', 'California', 'Alaska', 'Hawaii']
Midwest_states = ['Ohio', 'Indiana', 'Michigan', 'Illinois', 'Missouri', 
                  'Wisconsin', 'Minnesota', 'Iowa', 'Kansas', 'Nebraska', 'South Dakota', 'North Dakota']
Southwest_states = ['Texas', 'Oklahoma', 'New Mexico', 'Arizona']
Southeast_states = ['West Virginia', 'Virginia', 'Kentucky', 'Tennessee', 'North Carolina', 
                    'South Carolina', 'Georgia', 'Alabama', 'Mississippi', 'Arkansas', 'Louisiana', 'Florida']
Northeast_states = ['Maine', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New Hampshire', 'Vermont', 
                    'New York', 'Pennsylvania', 'New Jersey', 'Delaware', 'Maryland']

In [21]:
#use a function to get region information
def get_region(state):
    if state in west_states:
        return 'West'
    elif state in Midwest_states:
        return 'Midwest'
    elif state in Southwest_states:
        return 'Southwest'
    elif state in Southeast_states:
        return 'Southeast'
    else:
        return 'Northeast'

In [22]:
suicide_pop = suicide_pop.merge(state[['statefips', 'state_name', 'state']], on = ['statefips'], how = 'inner')
suicide_pop.head()

,year,statefips,fips,death_cause,pop,state_name,state
0,1960,01,01003,1,49088,Alabama,AL
1,1960,01,01007,2,14357,Alabama,AL
2,1960,01,01009,2,25449,Alabama,AL
3,1960,01,01011,1,13462,Alabama,AL
4,1960,01,01013,3,24560,Alabama,AL


In [23]:
suicide_pop['region'] = suicide_pop['state_name'].apply(get_region)
suicide_pop.head()

,year,statefips,fips,death_cause,pop,state_name,state,region
0,1960,01,01003,1,49088,Alabama,AL,Southeast
1,1960,01,01007,2,14357,Alabama,AL,Southeast
2,1960,01,01009,2,25449,Alabama,AL,Southeast
3,1960,01,01011,1,13462,Alabama,AL,Southeast
4,1960,01,01013,3,24560,Alabama,AL,Southeast


In [24]:
suicide_pop = suicide_pop.rename(columns = {'death_cause': 'deaths'})
suicide_pop['suicide_rate'] = (suicide_pop['deaths']/suicide_pop['pop'])*100000
suicide_pop.head()

,year,statefips,fips,deaths,pop,state_name,state,region,suicide_rate
0,1960,01,01003,1,49088,Alabama,AL,Southeast,2.037158
1,1960,01,01007,2,14357,Alabama,AL,Southeast,13.930487
2,1960,01,01009,2,25449,Alabama,AL,Southeast,7.858855
3,1960,01,01011,1,13462,Alabama,AL,Southeast,7.428317
4,1960,01,01013,3,24560,Alabama,AL,Southeast,12.214984


In [25]:
suicide_pop['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019])

### Load heatwave data

In [26]:
heatwave_count = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/climate_heatwave/heatwave_1960_2020/heatwave_definition2/heatwave_count_county_level.csv', dtype = {'fips': str})
heatwave_count = heatwave_count.loc[:, ~heatwave_count.columns.str.contains('^Unnamed')]
heatwave_count.head()

,fips,year,month,heatwave_count
0,01001,1960,6,1
1,01001,1960,7,2
2,01001,1962,7,1
3,01001,1963,6,1
4,01001,1963,8,1


In [27]:
heatwave_count['month'].unique()

array([ 6,  7,  8,  9, 10,  5, 12,  1, 11,  4,  3])

In [28]:
heatwave_year = heatwave_count.drop('month', axis = 1)
heatwave_year = heatwave_year.groupby(['fips', 'year']).sum()
heatwave_year = heatwave_year.reset_index()
heatwave_year.head()

,fips,year,heatwave_count
0,01001,1960,3
1,01001,1962,1
2,01001,1963,2
3,01001,1964,1
4,01001,1965,1


In [29]:
suicide_climate = suicide_pop.merge(heatwave_year, on = ['fips', 'year'], how = 'left')
suicide_climate.head()

,year,statefips,fips,deaths,pop,state_name,state,region,suicide_rate,heatwave_count
0,1960,01,01003,1,49088,Alabama,AL,Southeast,2.037158,NaN
1,1960,01,01007,2,14357,Alabama,AL,Southeast,13.930487,NaN
2,1960,01,01009,2,25449,Alabama,AL,Southeast,7.858855,NaN
3,1960,01,01011,1,13462,Alabama,AL,Southeast,7.428317,5.0
4,1960,01,01013,3,24560,Alabama,AL,Southeast,12.214984,3.0


In [30]:
suicide_climate['heatwave_count'] = suicide_climate['heatwave_count'].fillna(0)

In [31]:
suicide_climate['heatwave_count'].unique()

array([ 0.,  5.,  3.,  1.,  4.,  2.,  6.,  7.,  8.,  9., 10., 11., 12.,
       16., 13., 14., 15., 17.])

In [32]:
suicide_climate['heatwave_count'].describe()

count    152082.000000
mean          2.476026
std           1.974677
min           0.000000
25%           1.000000
50%           2.000000
75%           4.000000
max          17.000000
Name: heatwave_count, dtype: float64

In [33]:
south = suicide_climate[(suicide_climate['region'] == 'Southwest')|(suicide_climate['region'] == 'Southeast')]
south.head()

,year,statefips,fips,deaths,pop,state_name,state,region,suicide_rate,heatwave_count
0,1960,01,01003,1,49088,Alabama,AL,Southeast,2.037158,0.0
1,1960,01,01007,2,14357,Alabama,AL,Southeast,13.930487,0.0
2,1960,01,01009,2,25449,Alabama,AL,Southeast,7.858855,0.0
3,1960,01,01011,1,13462,Alabama,AL,Southeast,7.428317,5.0
4,1960,01,01013,3,24560,Alabama,AL,Southeast,12.214984,3.0


In [34]:
south['heatwave_count'].describe()

count    70749.000000
mean         2.369122
std          2.159344
min          0.000000
25%          1.000000
50%          2.000000
75%          4.000000
max         17.000000
Name: heatwave_count, dtype: float64

In [35]:
southeast = suicide_climate[suicide_climate['region'] == 'Southeast']
southeast.head()

,year,statefips,fips,deaths,pop,state_name,state,region,suicide_rate,heatwave_count
0,1960,01,01003,1,49088,Alabama,AL,Southeast,2.037158,0.0
1,1960,01,01007,2,14357,Alabama,AL,Southeast,13.930487,0.0
2,1960,01,01009,2,25449,Alabama,AL,Southeast,7.858855,0.0
3,1960,01,01011,1,13462,Alabama,AL,Southeast,7.428317,5.0
4,1960,01,01013,3,24560,Alabama,AL,Southeast,12.214984,3.0


In [36]:
southeast['heatwave_count'].describe()

count    53666.000000
mean         2.243581
std          2.079712
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         17.000000
Name: heatwave_count, dtype: float64

In [37]:
south.to_csv('monthly_GAM_heatwave_south_yearly_6020.csv')

In [38]:
southeast['heatwave_count'].unique()

array([ 0.,  5.,  3.,  1.,  4.,  2.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 17.])

In [39]:
southeast.to_csv('monthly_GAM_heatwave_southeast_yearly_6020.csv')

In [64]:
suicide_climate.shape

(48186, 11)

In [23]:
suicide_climate

,year,month,deaths,pop,suicide_rate,heatwave_count
0,2000,4,2485,253540949,0.980118,0
1,2000,8,2562,254358910,1.007238,128
2,2000,12,2225,255177896,0.871941,0
3,2000,2,2345,253131713,0.926395,0
4,2000,1,2599,252927063,1.027569,0
...,...,...,...,...,...,...
235,2019,3,4176,297886612,1.401876,0
236,2019,5,4057,297891200,1.361907,4
237,2019,11,3610,297905757,1.211793,1
238,2019,10,4055,297903483,1.361179,20


In [45]:
suicide_month = suicide_climate.drop(['suicide_rate', 'fips'], axis = 1)
suicide_month = suicide_month.groupby(['year', 'month']).sum()
suicide_month = suicide_month.reset_index()
suicide_month.head()

,year,month,deaths,pop,heatwave_count
0,2000,1,2475,188299253,0
1,2000,2,2257,185187954,0
2,2000,3,2451,185557502,0
3,2000,4,2375,188053452,0
4,2000,5,2516,190589472,4


### Load temperature and precipitation data from PRISM

In [24]:
data_temp = []
for year in years:
    temp_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/PRISM_Data/PRISM_daily_county_level/prism_daily_county_level_' + str(year) + '.csv', dtype = {'year': int, 'fips': str})
    temp_year = temp_year.loc[:, ~temp_year.columns.str.contains('^Unnamed')]
    data_temp.append(temp_year)

In [25]:
climate = pd.concat(data_temp)
climate.head()

,fips,date,tMean,tMin,tMax,prec,year
0,01001,2000-01-01,16.262459,10.089380,22.435537,0.560423,2000
1,01001,2000-01-02,17.412651,11.954220,22.871081,0.390251,2000
2,01001,2000-01-03,18.705264,14.001039,23.409490,0.216170,2000
3,01001,2000-01-04,16.203456,9.861904,22.545009,16.488676,2000
4,01001,2000-01-05,6.494604,-1.153744,14.142953,0.216055,2000


In [26]:
import datetime

In [27]:
def convertTime(time):
    return datetime.strptime(time, '%Y-%m-%d')

In [28]:
def getMonth(date):
    return(date.strftime('%b'))

In [29]:
climate['date'] = pd.to_datetime(climate['date'])
climate['month'] = climate['date'].apply(getMonth)

In [30]:
climate['month'] = climate['month'].replace(month_replace)
climate.head()

,fips,date,tMean,tMin,tMax,prec,year,month
0,01001,2000-01-01,16.262459,10.089380,22.435537,0.560423,2000,1
1,01001,2000-01-02,17.412651,11.954220,22.871081,0.390251,2000,1
2,01001,2000-01-03,18.705264,14.001039,23.409490,0.216170,2000,1
3,01001,2000-01-04,16.203456,9.861904,22.545009,16.488676,2000,1
4,01001,2000-01-05,6.494604,-1.153744,14.142953,0.216055,2000,1


In [46]:
climate = climate.drop(['statefips', 'state_name', 'state', 'date'], axis = 1)
climate = climate.groupby(['year', 'month', 'region']).mean()
climate = climate.reset_index()
climate.head()

,year,month,region,tMean,tMin,tMax,prec
0,2000,1,Midwest,24.131735,13.734550,34.528919,1.101389
1,2000,1,Northeast,24.386417,14.782550,33.990284,2.720459
2,2000,1,Southeast,41.503673,30.796313,52.211033,3.218615
3,2000,1,Southwest,47.132753,33.657913,60.607594,1.114261
4,2000,1,West,30.973741,21.495705,40.451777,3.315519


In [47]:
suicide_climate = suicide_pop.merge(climate, on = ['year', 'month', 'region'], how = 'inner')
suicide_climate.head()

,year,region,month,deaths,pop,suicide_rate,tMean,tMin,tMax,prec
0,2000,Midwest,4,536,58609705,0.914524,48.604590,35.948717,61.260462,2.009247
1,2000,Midwest,8,583,58662683,0.993817,73.528029,61.980313,85.075744,2.368660
2,2000,Midwest,12,471,58715987,0.802167,15.491818,6.364439,24.619197,1.339481
3,2000,Midwest,2,498,58583129,0.850074,33.237830,22.512694,43.962965,1.778046
4,2000,Midwest,1,584,58569836,0.997100,24.131735,13.734550,34.528919,1.101389


In [24]:
summer_winter = [6, 7, 8, 12, 1, 2]
def is_summer_winter(month):
    if month in summer_winter :
        return 1
    else:
        return 0

In [25]:
suicide_climate['summer_winter'] = suicide_climate['month'].apply(is_summer_winter)
suicide_climate.head()

,year,month,deaths,pop,suicide_rate,heatwave_count,summer_winter
0,2000,4,2485,253540949,0.980118,0,0
1,2000,8,2562,254358910,1.007238,128,1
2,2000,12,2225,255177896,0.871941,0,1
3,2000,2,2345,253131713,0.926395,0,1
4,2000,1,2599,252927063,1.027569,0,1


In [27]:
suicide_climate['heatwave_count'].describe()

count     240.000000
mean       98.437500
std       226.589569
min         0.000000
25%         0.000000
50%         1.000000
75%        69.750000
max      1450.000000
Name: heatwave_count, dtype: float64

In [26]:
suicide_climate.to_csv('monthly_GAM_heatwave_updated.csv')